In [5]:
import pandas as pd
import numpy as np
from pathlib import Path

data_folder = Path('/sgoinfre/goinfre/Perso/pdespres/data/')
#data_folder = Path('E:/Downloads/')
#data_folder = Path('C:/Users/Paul/Downloads/')

df = pd.read_csv(data_folder/'train2_irf_00010.csv', \
                dtype='int8')
y = pd.read_csv(data_folder/'y2.csv', \
               sep=';', usecols=[1], dtype='int8')

print(df.shape, y.shape)

(26495, 1511) (26495, 1)


In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold, cross_val_score
from sklearn.metrics import roc_auc_score
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt import space_eval
import warnings
from sklearn.externals import joblib

warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=PendingDeprecationWarning)

def Performance(Model,Y,X):
    AUC = roc_auc_score(Y, Model.predict_proba(X)[:,1])
    aucs.append(AUC)
    print ('the AUC is : %0.4f' %  AUC)
    return AUC

def auc_model(params):
    clf = LogisticRegression(**params)
    return cross_val_score(clf, train_x, train_y, scoring='roc_auc').mean()

def last_model(params):
    clf = LogisticRegression(**params)
    clf.fit(s1_x, s1_y)
    AUC = Performance(clf, valid_y, valid_x)
    return clf, AUC

param_space = {
   'random_state': 42,
   'C': 0.1,
    #'C': hp.uniform('C', 0.05, 0.5),
   'penalty': 'l1',
}

def f(params):
    global best
    global best_params
    auc = auc_model(params)
    if auc > best:
       best = auc
       best_params = params
    print (auc, params)
    return {'loss': -auc, 'status': STATUS_OK}

aucs = []

s1 = StratifiedShuffleSplit(n_splits=1, test_size=0.20, random_state=42)
for t1_index, val_index in s1.split(df, y):
    s1_x, s1_y = pd.DataFrame(df.iloc[t1_index]), y.iloc[t1_index].values.ravel()
    valid_x, valid_y = df.iloc[val_index], y.iloc[val_index].values.ravel()

#cv = StratifiedKFold(n_splits=4, shuffle=True, random_state=42)
cv = StratifiedShuffleSplit(n_splits=1, test_size=0.25, random_state=42)
for train_index, test_index in cv.split(s1_x, s1_y):
    train_x, train_y = pd.DataFrame(s1_x.iloc[train_index]), s1_y[train_index]
    test_x, test_y = s1_x.iloc[test_index], s1_y[test_index]

    trials = Trials()
    best = 0
    fmin(f, param_space, algo=tpe.suggest, max_evals=100, trials=trials)
    print ('\nbest:')
    model, AUC = last_model(best_params)
    print(best)
    print(AUC)
    output = 'LR_' + str(int(AUC * 10000)) + '.pkl'
    joblib.dump(model, data_folder/output)    
    #print (space_eval(param_space, best))
    print(best_params)
    print('\n')
#print(np.mean(aucs))

0.820805816062 {'C': 0.1, 'penalty': 'l1', 'random_state': 42}
0.820805816062 {'C': 0.1, 'penalty': 'l1', 'random_state': 42}
0.820805816062 {'C': 0.1, 'penalty': 'l1', 'random_state': 42}


KeyboardInterrupt: 

In [3]:
#0.8321 ENS
#0.8383 search 100
import pandas as pd
from sklearn.externals import joblib
from pathlib import Path

#data_folder = Path('/sgoinfre/goinfre/Perso/pdespres/data/')
data_folder = Path('E:/Downloads/')
#data_folder = Path('C:/Users/Paul/Downloads/')

classifier = joblib.load(data_folder/'LR_8383.pkl') 
t1 = pd.read_csv(data_folder/'irf_weight_00010.csv', names=['a','b'])
test = pd.read_csv(data_folder/'test_redux.csv')
t1.drop(['a'], axis=1, inplace=True)
t1 = t1.drop(t1.index[:1])
rfc_test = test.drop(t1['b'].tolist(), axis=1)
probas_ = classifier.predict_proba(rfc_test)[:, 1]
classifier.get_params()

{'C': 0.1,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'max_iter': 100,
 'multi_class': 'ovr',
 'n_jobs': 1,
 'penalty': 'l1',
 'random_state': None,
 'solver': 'liblinear',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [4]:
submission = pd.DataFrame(columns={'Ids', 'TARGET'})
submission['Ids'] = [i for i in range(26500,39750)]
submission['TARGET'] = probas_
submission = submission[['Ids', 'TARGET']]
submission.to_csv(data_folder/'submission.csv', sep=';', index=False, header={'Ids', 'TARGET'})